<a href="https://colab.research.google.com/github/jinseriouspark/commerce-llm/blob/main/extract_keywords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# extract- keywords
- 진행일자 : 24.03.26
- 목적 : 해당 리뷰를 분리할 수 있는 핵심 키워드를 추출
- 비교항목 :
- 판단 기준 : 1~5를 가장 잘 분리할 수 있는 모델


In [1]:
!pip install datasets
!pip install transformers[torch]
!pip install accelrate -U
!pip install seqeval


ERROR: Could not find a version that satisfies the requirement accelrate (from versions: none)
ERROR: No matching distribution found for accelrate


In [25]:
from sklearn.model_selection import train_test_split
import json
#!pip install datasets
from datasets import load_dataset, load_metric
import pandas as pd

In [ ]:
with open('/content/drive/MyDrive/commerce-llm/rawdata/naver_shopping.txt', 'r', encoding = 'utf-8') as f:
  data =f.readlines()

In [ ]:
rawdata =[i.rstrip().split('\t') for i in data]

In [ ]:
len(rawdata)

200000

- 20만개를 가지고 하기엔 1개 모델당 4시간 정도가 걸림
- 판단기준을 주어진 시간 내 잘 검증할 수 있도록 샘플 데이터를 아래와 같이 추출

In [ ]:
import pandas as pd
raw_df = pd.DataFrame(rawdata)

In [ ]:
raw_df.head()

,0,1
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [ ]:
features = raw_df[1]
target = raw_df[0]

In [ ]:
raw_df[0].value_counts() / raw_df.shape[0]
# 3번은 존재하지 않음
# 1 18%, 2 32%, 4 9.3%, 5 40%

5    0.405885
2    0.319945
1    0.180240
4    0.093930
Name: 0, dtype: float64

In [ ]:
min_cnt = raw_df[0].value_counts().min()
raw_df_for_test = raw_df.groupby(0).apply(lambda x : x.sample(min_cnt)).reset_index(drop = True)

raw_df_for_test.shape[0] / raw_df.shape[0] # 37% 만 가지고 테스트셋으로 만들어 진행?
raw_df_for_test.columns = ['target','review']
raw_df_for_test['target'] = raw_df_for_test['target'].astype(int)


X_train, X_test, _, _ = train_test_split(raw_df_for_test.values, raw_df_for_test['target'], stratify = raw_df_for_test['target'], random_state= 331)


In [ ]:
with open('/content/drive/MyDrive/commerce-llm/rawdata/rawdata_train.jsonl','w') as f:
  for row in X_train:
    json.dump({'rating': row[0], 'review': row[1]}, f, ensure_ascii = False)
    f.write('\n')

with open('/content/drive/MyDrive/commerce-llm/rawdata/rawdata_test.jsonl','w') as f:
  for row in X_test:
    json.dump({'rating': row[0], 'review': row[1]}, f, ensure_ascii = False)
    f.write('\n')

In [2]:
data_files = {'train': '/content/drive/MyDrive/commerce-llm/rawdata/rawdata_train.jsonl', 'test' : '/content/drive/MyDrive/commerce-llm/rawdata/rawdata_test.jsonl'}
review_dataset = load_dataset('json', data_files = data_files)

NameError: name 'load_dataset' is not defined

In [ ]:
review_dataset

In [ ]:
label_len = len(set(review_dataset['train']['rating']))

In [ ]:
# baseline : 만약 이 모든 값들을 그냥 분류한다면 어떻게 될까?
model_name = "monologg/kobert"
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments
import torch
import torch.nn as nn
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = label_len).to(device)

In [ ]:
def tokenize_function(row):
  return tokenizer(row['review'], padding = 'max_length', truncation = True)

# tokenize dataset
tokenized_datasets = review_dataset.map(tokenize_function, batched = True)

In [ ]:
tokenized_datasets

In [ ]:
from torch.utils.data import DataLoader
tokenized_datasets = tokenized_datasets.rename_column('rating','labels')
tokenized_datasets.set_format('torch', columns = ['input_ids','attention_mask','token_type_ids','labels'])
train_dataloader = DataLoader(tokenized_datasets['train'], shuffle = True, batch_size = 4)
test_dataloader = DataLoader(tokenized_datasets['test'], batch_size = 4)


In [ ]:
optimizer = optim.AdamW(model.parameters(), lr = 5e-5)
criterion = nn.CrossEntropyLoss()

In [ ]:
#!pip install seqeval
#!pip install datasets
from datasets import load_dataset, load_metric
import numpy as np

metric = load_metric('seqeval')
def compute_metrics(pred):
  predictions, labels = pred
  predictions = np.argmax(predictions, axis=2)

  # speical token은 삭제
  true_predictions = [
      [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(predictions, labels)
  ]

  true_labels = [
      [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(predictions, labels)
  ]

  # metrics.compute
  results = metric.compute(predictions = true_predictions, references = true_labels)
  flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
  }

  for k in results.keys():
    if k not in flattened_results.keys():
      flattened_results[f'{k}_f1'] = results[k]['f1']

  return flattened_results

In [ ]:
next(iter(train_dataloader))

In [ ]:
from tqdm import tqdm

num_epochs = 3
for epoch in range(num_epochs):
  model.train()
  total_loss = 0
  for batch in tqdm(train_dataloader):
    target = batch['labels'].to(device)
    batch = {k : v.to(device) for k, v in batch.items() if k!= 'labels'}
    outputs = model(input_ids = batch['input_ids'], attention_mask = batch['attention_mask'])
    loss = criterion(outputs.logits, target)
    total_loss += loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  avg_train_loss = total_loss / len(train_dataloader)
  print(f'Epoch {epoch+1} / {num_epochs} | Training Loss : {avg_train_loss}')

  # 검증
  model.eval()
  total_eval_loss = 0
  with torch.no_grad():
    for batch in test_dataloader:
      target = batch['labels'].to(device)
      batch = {k : v.to(device) for k, v in batch.items() if k!= 'labels'}
      outputs = model(input_ids = batch['input_ids'], attention_mask = batch['attention_mask'])

      loss = criterion(output.logtis, target)
      total_eval_loss += loss

  avg_eval_loss = total_eval_loss / len(test_dataloader)
  print(f'Epoch {epoch + 1} / {num_epochs} | Eval loss : {agvg_eval_loss}' )

model.save_pretrained(f'/content/drive/MyDrive/commerce-llm/model/model_{epoch+1}')

In [ ]:
loss = criterion(outputs.logits, target)

In [ ]:
loss

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
criterion(outputs.logits, target)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

class myDataset(Dataset):
  def __init__(self, text, label, tokenizer):
    self.text = text
    self.label = label
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.text)

  def __getitem__(self, idx):
    encoded_text = self.tokenizer(self.text[idx], return_tensors = 'pt', padding = True)
    label = torch.tensor(self.label[idx])
    return encoded_text, label

In [ ]:

ds = myDataset(list(raw_df_for_test['review']), list(raw_df_for_test['target']), tokenizer)

In [ ]:
dl = DataLoader(ds)

NameError: name 'DataLoader' is not defined

In [ ]:
raw_df_for_test.to_json('/content/drive/MyDrive/commerce-llm/rawdata/rawdata.jsonl', orient = 'records', lines = True)

In [ ]:
#!pip install datasets
from datasets import load_dataset, load_metric

In [ ]:
ds[:3]

NameError: name 'i' is not defined

# 비교 모델
- [참고자료](https://https://www.analyticsvidhya.com/blog/2022/03/keyword-extraction-methods-from-documents-in-nlp/)

1. Rake_NLTK
2. Spacy
3. Textrank
4. Word Cloud
5. KeyBert
6. Yet Another Keyword Extractor Yake
7. MonkeyLearn API
8. Textrazor API
9. applied NLI

In [ ]:
# KEYWORD EXTRACTION BY keybert

# KEYWORD EXTRACTION BY keybert
- keybert 란?

In [ ]:
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 751.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 k

In [ ]:
from keybert import KeyBERT
from tqdm import tqdm

model = KeyBERT(model="all-MiniLM-L6-v2")
keywords_list = []
for raw in tqdm(rawdata[:100]):
  keywords = model.extract_keywords(raw)
  keywords_list.append(keywords)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

100%|██████████| 100/100 [00:05<00:00, 17.55it/s]


In [ ]:
len(keywords_list)

578

In [ ]:
import pandas as pd
result_df = pd.DataFrame(data = {'origin' : rawdata[:len(keywords_list)], 'keywords': keywords_list})

In [ ]:
result_df.head()

,origin,keywords
0,"[5, 배공빠르고 굿]","[[], [(배공빠르고, 0.9021)]]"
1,"[2, 택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고]","[[], [(엉망이네용, 0.7305), (저희집, 0.6034), (놔두고가고, ..."
2,"[5, 아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이...","[[], [(엉성하긴, 0.7211), (대박입니다, 0.6205), (가성비, 0..."
3,"[2, 선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니...","[[], [(사는거, 0.7021), (머그컵만, 0.6322), (확인안하고, 0..."
4,"[5, 민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ]","[[], [(민트색상, 0.7763), (손잡이는, 0.7663), (용도로도, 0..."


In [ ]:
# extrack keywords with mistral 7b
# https://iamshobhitagarwal.medium.com/running-mistral-7b-large-language-model-on-google-colab-gpu-completely-free-of-cost-fc0a57279de9

In [ ]:
!git lfs install
!git clone https://huggingface.co/mistralai/Mistral-7B-v0.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q torch datasets
!pip install -q accelerate==0.21.0
!pip install -q peft==0.4.0
!pip install -qbitsandbytes==0.40.2
!pip install -q transformers==4.31.0
!pip install -q trl==0.4.7

     ━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/23.7 MB 18.7 MB/s eta 0:00:02ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device

     ━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/23.7 MB 21.1 MB/s eta 0:00:01
     ━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/23.7 MB 19.9 MB/s eta 0:00:01ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device

     ━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/23.7 MB 19.9 MB/s eta 0:00:01
     ━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/23.7 MB 18.2 MB/s eta 0:00:01
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/urllib3/response.py", line 527, in 

In [ ]:
!nvidia-smi

Wed Mar 27 12:44:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

In [ ]:
model_path= '/content/Mistral-7B-v0.1'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path,
                                             trust_remote_code = True,
                                             device_map = 'auto',
                                             load_in_8bit = True,
                                             max_memory = {0:"15GB"})


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [ ]:
pip install accelerate

  Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)
     ━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/23.7 MB 16.0 MB/s eta 0:00:02
ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



In [ ]:
model.eval()

NameError: name 'model' is not defined